In [1]:
### mmdetection에 필요한 좌표 형식 (coco dataset 기준) [top left x position, top left y position, width, height]
### class  
### 01_ulcer,02_mass,04_lymph,05_bleeding 총 4 개  

In [2]:
from mmdet.apis import init_detector, inference_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed

from custom_data_pipeline import CUSTOM_LoadImageFromFile


import json

In [3]:
@DATASETS.register_module()
class lesion_ds(CustomDataset):
    CLASSES=('01_ulcer','02_mass','04_lymph', '05_bleeding')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = { '01_ulcer' : 0 , '02_mass' : 1, '04_lymph' : 2, '05_bleeding' : 3}
        
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
        
        data_infos = []
        
        for idx,img in enumerate(image_list):
            json_data = {}
            with open(img, "r") as json_file:
                json_data = json.load(json_file)
            
            filename = img # json에 annotation + image라서 json 자체를 filename로 주고 LoadImageFromFile을 baseline을 참조하여 custom으로 

            height = json_data['imageHeight']
            width = json_data['imageWidth']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for a_idx in range(len(json_data['shapes'])):
                gt_labels.append(CLASSES_dict[json_data['shapes'][a_idx]['label']])
                
                ## 좌표순서 좌상 우상 우하 좌하 
                ## mmdetection의 default annotation loader는 'coco_panoptic.py'에 있는데 여기 기준으로 하면 bbox에는 x1,y1,x2,y2가 담겨야함
                ## 만약에 이렇게 따로 load를 하지 않을시에는 x,y,w,h 형태로 annotaion을 생성하면 자동으로 x1,y1,x2,y2로 변환해줌 
                ori_pos = np.array(json_data['shapes'][a_idx]['points'])
                x1, y1, x2, y2 = min(ori_pos[:, 0]), min(ori_pos[:, 1]), max(ori_pos[:, 0]), max(ori_pos[:, 1])
                
                if x1 == 0 : 
                    x1 = 1 
                if y1 == 0 : 
                    y1 = 1 
                
                if x2 == width: 
                    x2 = x2 - 1 
                if y2 == width: 
                    y2 = y2-1 
            
                
                if x1==x2 or y1==y2:
                    print('Grond-truth Bounding Box 이상체크')
                    print(filename)
                     
                
                gt_bboxes.append([x1,y1,x2,y2])
                

            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(image_list))+' load annotations END!')
            
        
        
        return data_infos
    

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('mmdetection/configs/deformable_detr/deformable_detr_twostage_refine_r50_16x2_50e_coco.py') 

In [5]:
cfg.dataset_type  = 'lesion_ds'
cfg.data_root = ''

## coco datset으로 pretrain된 weight load 
## 혹시 50epoch 이후에도 validation mAP 올라가나 싶어서 더돌려봄 ==> 안올라가서 배제
# cfg.load_from = 'lesion_checkpoints_ver1/epoch_50.pth'
# pretrain weights
cfg.load_from = 'pretrain/deformable_detr_twostage_refine_r50_16x2_50e_coco.pth'

cfg.work_dir = 'lesion_checkpoints_ver1'

cfg.evaluation.metric = 'mAP'

#class 갯수 매칭 
cfg.model.bbox_head.num_classes=4

#데이터 pipeline은 config 참조 
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
# train_pipeline, NOTE the img_scale and the Pad's size_divisor is different
# from the default setting in mmdet.

## 원본이 576 / 576
## 이미지의 높이 너비 비율은 동일하게 두는 것으로..
## 기존 pretrain시에도 Auto augment사용한 것을 비율만 다르게 해서 그대로 사용 
cfg.train_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='AutoAugment',
        policies=[
            [
                dict(
                    type='Resize',
                    img_scale=[(480, 480), (512, 512), (544, 544),
                               (576, 576), (608, 608), (640, 640),
                               (672, 672), (704, 704), (736, 736)
                              ],
                    multiscale_mode='value',
                    keep_ratio=True)
            ],
            [
                dict(
                    type='Resize',
                    # The radio of all image in train dataset < 7
                    # follow the original impl
                    img_scale=[(400, 400), (500, 500), (600, 600)],
                    multiscale_mode='value',
                    keep_ratio=True),
                dict(
                    type='RandomCrop',
                    crop_type='absolute_range',
                    crop_size=(384, 384),
                    allow_negative_crop=True),
                dict(
                    type='Resize',
                    img_scale=[(480, 480), (512, 512), (544, 544),
                               (576, 576), (608, 608), (640, 640),
                               (672, 672), (704, 704), (736, 736)
                               ],
                    multiscale_mode='value',
                    override=True,
                    keep_ratio=True)
            ]
        ]),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size_divisor=1),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

# test_pipeline, NOTE the Pad's size_divisor is different from the default
# setting (size_divisor=32). While there is little effect on the performance
# whether we use the default setting or use size_divisor=1.
cfg.test_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(736, 736),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='Pad', size_divisor=1),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

# batch size 3060에 맞게 set (기존 pretrain시에는  batch 32 사용했다고 되어있음)
cfg.data = dict(
    samples_per_gpu=4,
    workers_per_gpu=2,
    train=dict(filter_empty_gt=False, pipeline=cfg.train_pipeline,ann_file='splits/lesion_train.txt'),
    val=dict(pipeline=cfg.test_pipeline,ann_file='splits/lesion_val.txt'),
    test=dict(pipeline=cfg.test_pipeline))


cfg.data.train.type = cfg.dataset_type
cfg.data.val.type = cfg.dataset_type
cfg.data.test.type = cfg.dataset_type

cfg.model.neck.norm_cfg = dict(type='GN', num_groups=4) #batch size 매칭 


#2478
cfg.log_config.interval = 1500 #iteration 단위
cfg.evaluation.interval = 1 #epoch 단위
cfg.checkpoint_config.interval = 1 #epoch 단위

# seed set
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'lesion_ds'
data_root = ''
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='CUSTOM_LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='AutoAugment',
        policies=[[{
            'type':
            'Resize',
            'img_scale': [(480, 480), (512, 512), (544, 544), (576, 576),
                          (608, 608), (640, 640), (672, 672), (704, 704),
                          (736, 736)],
            'multiscale_mode':
            'value',
            'keep_ratio':
            True
        }],
                  [{
                      'type': 'Resize',
                      'img_scale': [(400, 400), (500, 500), (600, 600)],
                      'multiscale_mode': 'value',
                      'keep_ratio': True
                  }, {
                      'type': 'RandomCrop',
       

In [6]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import torch

In [7]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

train_detector(model, datasets, cfg, distributed=False, validate=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


20000/59491 load annotations END!
40000/59491 load annotations END!


/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `feedforward_channels` in BaseTransformerLayer has been deprecated, now you should set `feedforward_channels` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `ffn_dropout` in BaseTransformerLayer has been deprecated, now you should set `ffn_drop` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `ffn_num_fcs` in BaseTransformerLayer has been deprecated, now you should set `num_fcs` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/opt/conda/lib/python3.7/site-packages/mmcv/cnn/brick

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3131/3131, 12.6 task/s, elapsed: 248s, ETA:     0s
---------------iou_thr: 0.5---------------


2021-12-05 15:10:29,751 - mmdet - INFO - 
+-------------+------+--------+--------+-------+
| class       | gts  | dets   | recall | ap    |
+-------------+------+--------+--------+-------+
| 01_ulcer    | 1918 | 212022 | 0.951  | 0.761 |
| 02_mass     | 494  | 15265  | 0.974  | 0.905 |
| 04_lymph    | 548  | 18462  | 0.929  | 0.794 |
| 05_bleeding | 995  | 67351  | 0.931  | 0.717 |
+-------------+------+--------+--------+-------+
| mAP         |      |        |        | 0.794 |
+-------------+------+--------+--------+-------+
2021-12-05 15:10:29,758 - mmdet - INFO - Epoch(val) [1][3131]	AP50: 0.7940, mAP: 0.7945
2021-12-05 15:27:29,475 - mmdet - INFO - Epoch [2][1500/14873]	lr: 2.000e-04, eta: 5 days, 5:57:57, time: 0.680, data_time: 0.007, memory: 9262, enc_loss_cls: 0.2629, enc_loss_bbox: 0.4173, enc_loss_iou: 0.2863, loss_cls: 0.2368, loss_bbox: 0.3979, loss_iou: 0.2677, d0.loss_cls: 0.3902, d0.loss_bbox: 0.3717, d0.loss_iou: 0.2599, d1.loss_cls: 0.2580, d1.loss_bbox: 0.3945, d1.los

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3131/3131, 12.6 task/s, elapsed: 249s, ETA:     0s
---------------iou_thr: 0.5---------------


2021-12-05 23:49:38,545 - mmdet - INFO - 
+-------------+------+--------+--------+-------+
| class       | gts  | dets   | recall | ap    |
+-------------+------+--------+--------+-------+
| 01_ulcer    | 1918 | 178463 | 0.943  | 0.721 |
| 02_mass     | 494  | 37026  | 0.974  | 0.900 |
| 04_lymph    | 548  | 49343  | 0.936  | 0.800 |
| 05_bleeding | 995  | 48268  | 0.899  | 0.715 |
+-------------+------+--------+--------+-------+
| mAP         |      |        |        | 0.784 |
+-------------+------+--------+--------+-------+
2021-12-05 23:49:38,552 - mmdet - INFO - Epoch(val) [4][3131]	AP50: 0.7840, mAP: 0.7836
2021-12-06 00:06:37,714 - mmdet - INFO - Epoch [5][1500/14873]	lr: 2.000e-04, eta: 4 days, 21:28:23, time: 0.679, data_time: 0.007, memory: 9262, enc_loss_cls: 0.2552, enc_loss_bbox: 0.4095, enc_loss_iou: 0.2835, loss_cls: 0.2221, loss_bbox: 0.3989, loss_iou: 0.2678, d0.loss_cls: 0.3915, d0.loss_bbox: 0.3673, d0.loss_iou: 0.2599, d1.loss_cls: 0.2460, d1.loss_bbox: 0.3948, d1.lo

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3131/3131, 12.6 task/s, elapsed: 248s, ETA:     0s
---------------iou_thr: 0.5---------------


2021-12-06 02:42:39,738 - mmdet - INFO - 
+-------------+------+--------+--------+-------+
| class       | gts  | dets   | recall | ap    |
+-------------+------+--------+--------+-------+
| 01_ulcer    | 1918 | 226264 | 0.961  | 0.753 |
| 02_mass     | 494  | 27056  | 0.984  | 0.928 |
| 04_lymph    | 548  | 20726  | 0.929  | 0.796 |
| 05_bleeding | 995  | 39054  | 0.893  | 0.709 |
+-------------+------+--------+--------+-------+
| mAP         |      |        |        | 0.797 |
+-------------+------+--------+--------+-------+
2021-12-06 02:42:39,746 - mmdet - INFO - Epoch(val) [5][3131]	AP50: 0.7970, mAP: 0.7965


KeyboardInterrupt: 